# Collecte, Traitement, et Analyse de données de réseaux sociaux

Importation des librairies

In [29]:
import json
import re
import pandas as pd
from textblob import TextBlob
import random as rd

Ouverture du fichier Json et nettoyage du text

In [ ]:
#Ouverture du fichier jason
with open('versailles_tweets_100.json') as json_data:
    data_dict = json.load(json_data)

#Effacer tous les caractères spéciaux dans le texte et récupérer le texte propre dans une liste
liste_txt =[]
for dict in data_dict:
    dict["text"] = re.sub(r'[\W_]+',' ', dict["text"])
    liste_txt.append(dict['text'])

# Création et écriture dans le fichier zone d'atterissage
with open("zone_d'atterrissage.json", 'w') as f2:
    json.dump(data_dict, f2)





Identification de l'auteur de la publication

In [3]:
with open('versailles_tweets_100.json') as json_data:
    data = json.load(json_data)

liste_auteurs= []
for dict in data:
    liste_auteurs.append(dict['author_id'])
print(liste_auteurs)

['1339914264522461187', '1339914264522461187', '1339914264522461187', '1339914264522461187', '717025418', '992904738516717570', '992904738516717570', '736523371', '1471684208', '992904738516717570', '3169236915', '992904738516717570', '16267684', '60117154', '3169236915', '372993152', '372993152', '105241852', '2357913366', '717025418']


Extraction de la liste des hashtags et la liste des mentions de chaque publication

In [4]:
liste_hashtags = []
liste_mentions = []

for dict in data:
    #Extraction des hashtags
    hashtags = re.findall(r'\B#\w*[a-zA-Z]+\w*', dict['text'])
    liste_hashtags.append(hashtags)
    #Extraction des mentions
    mentions = re.findall(r'\B@\w*[a-zA-Z]+\w*', dict['text'])
    liste_mentions.append(mentions)

print(liste_hashtags)

print(liste_mentions)

[['#twitter225'], ['#SupportriceMazo', '#domie', '#CIV'], ['#CIV'], ['#jifa'], [], [], [], [], ['#versailles', '#nocturne', '#appollon'], [], [], [], [], [], [], [], [], [], [], []]
[[], ['@ericbailly24', '@maxigr04del'], [], [], [], ['@isabelle170516', '@leonna_julie', '@Steiner2502'], ['@LynLyna12', '@leonna_julie'], [], [], ['@leonna_julie'], ['@miliemelo82', '@kilianbridoux', '@LeMeneec'], ['@Polo82810715', '@lrestistant73'], [], [], ['@Pauluskupa'], ['@anniemacmanus'], ['@yebbasmith', '@anniemacmanus'], [], ['@AzmiAnees3'], []]


Analyse de sentiments de la publication

In [27]:
liste_sentiments = []
with open("zone_d'atterrissage.json") as f:
   clean_data  = json.load(f)
   
for dico in clean_data:
    sentiment = TextBlob(dico['text']).sentiment 
    liste_sentiments.append(sentiment[1])
print(liste_sentiments)


[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Identification du/des topics de la publication 

In [33]:
# On va attribuer des topics aléatoirement 
liste_topics = ['sport', 'politique', 'culture', 'humour']
publication_topics = [rd.choice(liste_topics) for i in range(len(liste_auteurs))]
print(publication_topics)

['politique', 'sport', 'humour', 'sport', 'politique', 'humour', 'sport', 'politique', 'humour', 'culture', 'politique', 'sport', 'politique', 'humour', 'humour', 'culture', 'culture', 'sport', 'sport', 'humour']


Création du DataFrame à exploiter

In [43]:
df_exploitation = pd.DataFrame(list(zip(liste_auteurs,liste_txt, liste_hashtags, liste_mentions, liste_sentiments, publication_topics)), columns = ['ID', 'Texte','Hashtags', 'Mentions', 'Sentiments', 'Topics'])
df_exploitation['Hashtags'] = df_exploitation['Hashtags'].str.get(0)
df_exploitation['Mentions'] = df_exploitation['Mentions'].str.get(0)
df_exploitation


,ID,Texte,Hashtags,Mentions,Sentiments,Topics
0,1339914264522461187,Goumin des éléphants joueurs la même fatigue m...,#twitter225,NaN,0.0,politique
1,1339914264522461187,ericbailly24 maxigr04del mes tontons vous ave...,#SupportriceMazo,@ericbailly24,1.0,sport
2,1339914264522461187,Ah oui le sommeil là sera compliqué CIV est él...,#CIV,NaN,0.0,humour
3,1339914264522461187,31 juillet journée internationale de la femme ...,#jifa,NaN,0.0,sport
4,717025418,Le pedigree https t co D3Rv7A2BOF,NaN,NaN,0.0,politique
5,992904738516717570,isabelle170516 leonna julie Steiner2502 Vous ...,NaN,@isabelle170516,1.0,humour
6,992904738516717570,LynLyna12 leonna julie La grande muette conti...,NaN,@LynLyna12,0.0,sport
7,736523371,Under wsh,NaN,NaN,0.0,politique
8,1471684208,Les bains d apollon versailles nocturne appoll...,#versailles,NaN,0.0,humour
9,992904738516717570,leonna julie Le rdv aujourd hui aura tenu ses...,NaN,@leonna_julie,0.0,culture


Top K hashtags

In [ ]:
# création d'un dictionnaire contenant un hashtag et le nombre de fois qu'il apparaît
def K_hashtags(liste_hashtags):
    occurences = []
    dic = {}
    for i in range (len(liste_hashtags)):
            dic[liste_hashtags[i]] = liste_hashtags.count(liste_hashtags[i]) 
    return dic

K_hashtags(liste_hashtags)


Top K utilisateurs mentionnés

In [ ]:
# création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il est mentionné

def K_mention(liste_mentions):
    occurences = []
    dic = {}
    for i in range (len(liste_mentions)):
            dic[liste_mentions[i]] = liste_mentions.count(liste_mentions[i]) 
    return dic

K_mention(liste_mentions)


Top K utilisateur

In [ ]:
# création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il a posté un tweet
def K_utilisateur(liste_auteurs):
    occurences = []
    dic = {}
    for i in range (len(liste_auteurs)):
            dic[liste_auteurs[i]] = liste_auteurs.count(liste_auteurs[i]) 
    return dic

K_utilisateur(liste_auteurs)